## Import package and Setting

In [1]:
import os
import numpy as np
import tensorflow as tf
import random
import pathlib
import tensorflow_addons as tfa
from  efficientnet import tfkeras
from IPython.core.display import display, HTML 

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "8,9"
gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_memory_growth(gpus[1], True)
display(HTML("<style>.container { width:98% !important; }</style>")) 

## Setting Distribute Strategy

In [3]:
strategy = tf.distribute.MirroredStrategy()

In [4]:
print("Number of GPU: ", strategy.num_replicas_in_sync)

Number of GPU:  2


In [5]:
BATCH_SIZE_PER_REPLICA = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

## Load image data

In [6]:
data_path = "flower_photos/"

In [7]:
data_root = pathlib.Path(data_path)

In [8]:
label_names = sorted(item.name for item in data_root.glob("*/") if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))
print("label names:", label_names)
print("label to index:", label_to_index)

label names: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
label to index: {'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [9]:
all_image_paths = list(data_root.glob("*/*"))
all_image_paths = [str(path) for path in all_image_paths]

In [10]:
random.shuffle(all_image_paths)

In [11]:
train_image_paths = all_image_paths[500:]
valid_image_paths = all_image_paths[:500]

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [13]:
def preprocessing_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, tf.random.uniform([2], minval=224, maxval=256, dtype=tf.int32))
    image = tf.image.random_crop(image ,[224, 224, 3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tfa.image.rotate(image, tf.random.uniform([], -30, 30))
    
    image -= tf.reduce_mean(image)
    image /= (tf.math.reduce_std(image)+1e-6)
    return image

In [14]:
def load_and_preprocessing_image(path):
    image = tf.io.read_file(path)
    return preprocessing_image(image)

In [15]:
def dataset_generate(image_paths, batch_size, label_to_index, AUTOTUNE):
    image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in image_paths]
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    image_ds = path_ds.map(load_and_preprocessing_image, num_parallel_calls=AUTOTUNE)
    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(image_labels, tf.float32))
    image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
    image_counts = len(image_paths)
    print("Step:", int(image_counts/batch_size))
    image_label_ds = image_label_ds.shuffle(buffer_size=image_counts).batch(batch_size).prefetch(buffer_size=AUTOTUNE)
    return image_label_ds

In [16]:
train_datasets = dataset_generate(train_image_paths, 16, label_to_index, AUTOTUNE)
valid_datasets = dataset_generate(valid_image_paths, 16, label_to_index, AUTOTUNE)

Step: 198
Step: 31


In [17]:
train_dist_dataset = strategy.experimental_distribute_dataset(train_datasets)
valid_dist_dataset = strategy.experimental_distribute_dataset(valid_datasets)

## Build Model

In [18]:
def create_model():
    model = tf.keras.Sequential()
    conv_base = tfkeras.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
    model.add(conv_base)
    model.add(tf.keras.layers.GlobalAvgPool2D())
    model.add(tf.keras.layers.Dense(5, activation="softmax"))
    
    return model

In [19]:
with strategy.scope():
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    def compute_loss(label, predictions):
        per_example_loss = loss_object(label, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

In [20]:
with strategy.scope():
    valid_loss = tf.keras.metrics.Mean(name="valid_loss")
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
    valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="valid_accuracy")

In [21]:
with strategy.scope():
    model = create_model()
    optimizer = tf.keras.optimizers.Adam()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [22]:
with strategy.scope():
    def train_step(inputs):
        images, labels = inputs
        
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            loss = compute_loss(labels, predictions)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        train_accuracy.update_state(labels, predictions)
        
        return loss
    
    def valid_step(inputs):
        images, labels = inputs
        
        predictions = model(images, training=False)
        t_loss = loss_object(labels, predictions)
        
        valid_loss.update_state(t_loss)
        valid_accuracy.update_state(labels, predictions)

In [23]:
with strategy.scope():
    
    @tf.function
    def distributed_train_step(dataset_inputs):
        per_replica_losses = strategy.experimental_run_v2(train_step, args=(dataset_inputs, ))
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)
    
    @tf.function
    def distributed_valid_step(dataset_inputs):
        return strategy.experimental_run_v2(valid_step, args=(dataset_inputs, ))
    
    for epoch in range(30):
        total_loss = 0.0
        num_batches = 0
        for x in train_dist_dataset:
            total_loss += distributed_train_step(x)
            num_batches += 1
        train_loss = total_loss / num_batches
        
        for x in valid_datasets:
            distributed_valid_step(x)
            
        template = ("Epoch {}, Loss: {:.3f}, Accuracy: {:.3f}, Test Loss: {:.3f}, "
                "Test Accuracy: {:.3f}")
        print (template.format(epoch+1, train_loss,
                               train_accuracy.result()*100, valid_loss.result(),
                               valid_accuracy.result()*100))

        valid_loss.reset_states()
        train_accuracy.reset_states()
        valid_accuracy.reset_states()

INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 213 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
Epoch 1, Loss: 0.319, Accuracy: 76.341, Test Loss: 0.664, Test Accuracy: 81.200
Epoch 2, Loss: 0.230, Accuracy: 83.281, Test Loss: 0.471, Test Accuracy: 86.600
Epoch 3, Loss: 0.184, Accuracy: 87.476, Test Loss: 0.450, Test Accuracy: 87.300
Epoch 4, Loss: 0.161, Accuracy: 87.918, Test Loss: 0.504, Test Accuracy: 82.400
Epoch 5, Loss: 0.168, Accuracy: 87.571, Test Loss: 0.453, Test Accuracy: 84.700
Epoch 6, Loss: 0.143, Accuracy: 89.211, Test Loss: 0.341, Test Accuracy: 89.000
Epoch 7, Loss: 0.147, Accuracy: 89.716, Test Loss: 0.492, Tes

## Save model

In [24]:
model.save("flower_efficientnet.h5", include_optimizer=False)